# PSPNet 네트워크 구성 및 구현:

학습 목표:

1. PSPNet 네트워크 구조를 모듈 단위로 이해한다.
2. PSPNet을 구성하는 각 모듈의 역할을 이해한다.
3. PSPNet 네트워크 클래스의 구현을 이해한다.

PSPNet은 네 개의 모듈인 Feature, Pyramid Pooling Decoder, AuxLoss로 구성되었다.

PSPNet의 첫 번째 모듈은 Feature 모듈이며 Encoder 모듈로도 불린다. 입력 화상의 특징을 파악하는 것이 Feature 모듈의 목적이다. 

PSPNet의 두번 째 모듈은  Pyramid Pooling 모듈이다. PSPNet의 독창성을 보여주는 모듈이다. Pyramid Pooling 모듈로 해결하려는 문제는 '어떠한  픽셀의 물체 라벨을 구하려면 다양한 크기로 해당 픽셀 주변 정보가 필요'하다이다. 
하나의 픽셀만을 통해서는 해당 클래스의 라벨 값을 구분하기 어렵지만 해당 픽셀 주위를 점진적으로 확대한 특징량을 확인하면 소인지 돼지인지 알 수 있다는 것이다.

Pyramid  Pooling은 네가지 크기의 특징량 맵을 준비한다. 화상 전첼르 차지하는 특징량, 화상 절반을 차지하는 특징량, 화상의 1/3을 차지하는 특징량, 화상의 1/6을 차지하는 특징량으로 Feature 모듈의 출력을 처리한다. 

PSPNet의 세번 째 모듈은  Decoder 모듈이다. 업샘플링 모듈이라고도 한다. Decoder 모듈의 목적은 두가지이다. 첫째, Pyramid Pooling 모듈의 출력을 텐서로 변환하는 것이다. 둘째, 변환된 텐서를 원 입력  화상 크기에 맞도록 변환하는 것이다.

원래대로라면 위 세 개의 모듈로 시맨틱  분할이 이루어지지만 PSPNet에서는 네트워크 결합  파라미터의 학습을 더 잘 수행하기 위해 AuxLoss 모듈을  준비한다. Aux란 Auxilary의 약어로 보조라는 뜻이다.  AuxLoss 모듈은 손실함수 계산을 보조한다. 

Feature 모듈로 중간 텐서를 빼내 입력 데이터로 한다. 이를 Decoder 모듈처럼  각 픽셀에 대응하는 물체 라벨 추정 클래스 분류를 실행한다. AuxLoss 모듈의 입력 데이터 크기는 1024by60by60, 출력은 Decoder 모듈의 클래스 분류와 마찬가지로 21by475by475이다.

신경망 학습 시에는 AuxLoss 모듈의 출력과 Decoder 모듈의 출력을 모두 화상의 어노테이션 데이터로 대응시켜 손실 값을 계산한다. 게산 후 손실 값에 따른 오차 역전파 법을 실시하여 네트워크의 결합 파라미터를 갱신한다.

학습 시에는 AuxLoss 모듈을 사용하지만 추론 시에는 AuxLoss 모듈의 출력은 사용하지 않고 Decoder 모듈의 출력만으로 시맨틱 분할을 한다.

# PSPNet 클래스 구현:

Feature 모듈은 5개의 서브 네트워크 feature_conv, feature_res_1, feature_res_2, feature_dilated_res_1, feature_dilated_res_2로 하나의 서브 네트워크로 구성된다.

1. 서브 네트워크 FeatureMap_convolution:

서브 네트워크 FeatureMap_convolution은 단순히 합성곱, 배치 정규화, 최대 풀링으로 화상의 특징량을 추출하는 역할을 한다.

2. 서브 네트워크 ResidualBlockPSP:

ResidualBlockPSP는 ResidualNetwork 신경망에서 사용되는 ResidualBlock 구조를 사용한다.
ResidualBlockPSP는 bottleNeckPSP 클래스를 지나 bottleNeckIdentifyPSP 클래스를  여러 번 반복하여 출력한다.

3. bottleNeckPSP와 bottleNeckIdentifyPSP:

bottleNeckPSP와 bottleNeckIdentifyPSP 두 클래스는 특징적인 네트워크 구조를 가졌으며 입력이 두 갈래로 나위어 처리된다. 이렇게 두 갈래로 나누어진 입력 아래쪽 루트를 스킵 결합이라고 한다.

딥러닝에서는 네트워크가 깊어지면 열화 문제가 발생한다. 열화문제를 피하기 위해 Residual Block의 입력 x를 그대로 출력하는 스킵 결합을 먼저 준비한다. 입력 그대로 출력(아래쪽)을 x, 스킵을 하지 않고 진행한 출력(위쪽)을 F(x)라 하면 Residual Block 출력 y는 y=x+F(X)가 된다.

위쪽 경로의 각 유닛 학습 파라미터 값이 모두 0이라고 하면 Residual Block의 출력 y는 입력과 같은 x가 된다. 이 경우 Block이 여러 개 쌓여도 열화 문제를 피할 수 있다.

Residual Block은 Block의 출력y를 학습시키는 것이 아니라 들어 온 입력 x는 그대로  스킵 결합으로 출력시키고 y-x, 즉 원하던 출력과 들어온 입려과의 잔차인 y-x=F(x)를 배우게 하는 전략이 Residual Block이다. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class PSPNet(nn.Module):
    def __init__(self, n_classes):
        super(PSPNet, self).__init__()

        block_config = [3, 4, 6, 3]  
        img_size = 475
        img_size_8 = 60  

        self.feature_conv = FeatureMap_convolution()
        self.feature_res_1 = ResidualBlockPSP(
            n_blocks=block_config[0], in_channels=128, mid_channels=64, out_channels=256, stride=1, dilation=1)
        self.feature_res_2 = ResidualBlockPSP(
            n_blocks=block_config[1], in_channels=256, mid_channels=128, out_channels=512, stride=2, dilation=1)
        self.feature_dilated_res_1 = ResidualBlockPSP(
            n_blocks=block_config[2], in_channels=512, mid_channels=256, out_channels=1024, stride=1, dilation=2)
        self.feature_dilated_res_2 = ResidualBlockPSP(
            n_blocks=block_config[3], in_channels=1024, mid_channels=512, out_channels=2048, stride=1, dilation=4)

        self.pyramid_pooling = PyramidPooling(in_channels=2048, pool_sizes=[
            6, 3, 2, 1], height=img_size_8, width=img_size_8)

        self.decode_feature = DecodePSPFeature(
            height=img_size, width=img_size, n_classes=n_classes)

        self.aux = AuxiliaryPSPlayers(
            in_channels=1024, height=img_size, width=img_size, n_classes=n_classes)

    def forward(self, x):
        x = self.feature_conv(x)
        x = self.feature_res_1(x)
        x = self.feature_res_2(x)
        x = self.feature_dilated_res_1(x)

        output_aux = self.aux(x)  

        x = self.feature_dilated_res_2(x)

        x = self.pyramid_pooling(x)
        output = self.decode_feature(x)

        return (output, output_aux)


In [ ]:
class conv2DBatchNormRelu(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation, bias):
        super(conv2DBatchNormRelu, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels,
                              kernel_size, stride, padding, dilation, bias=bias)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.batchnorm(x)
        outputs = self.relu(x)

        return outputs

In [ ]:
class FeatureMap_convolution(nn.Module):
    def __init__(self):
     
        super(FeatureMap_convolution, self).__init__()

        in_channels, out_channels, kernel_size, stride, padding, dilation, bias = 3, 64, 3, 2, 1, 1, False
        self.cbnr_1 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size, stride, padding, dilation, bias)

        in_channels, out_channels, kernel_size, stride, padding, dilation, bias = 64, 64, 3, 1, 1, 1, False
        self.cbnr_2 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size, stride, padding, dilation, bias)

        in_channels, out_channels, kernel_size, stride, padding, dilation, bias = 64, 128, 3, 1, 1, 1, False
        self.cbnr_3 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size, stride, padding, dilation, bias)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.cbnr_1(x)
        x = self.cbnr_2(x)
        x = self.cbnr_3(x)
        outputs = self.maxpool(x)
        return outputs


In [ ]:
class ResidualBlockPSP(nn.Sequential):
    def __init__(self, n_blocks, in_channels, mid_channels, out_channels, stride, dilation):
        super(ResidualBlockPSP, self).__init__()


        self.add_module(
            "block1",
            bottleNeckPSP(in_channels, mid_channels,
                          out_channels, stride, dilation)
        )

        for i in range(n_blocks - 1):
            self.add_module(
                "block" + str(i+2),
                bottleNeckIdentifyPSP(
                    out_channels, mid_channels, stride, dilation)
            )

In [ ]:
class conv2DBatchNorm(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation, bias):
        super(conv2DBatchNorm, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels,
                              kernel_size, stride, padding, dilation, bias=bias)
        self.batchnorm = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv(x)
        outputs = self.batchnorm(x)

        return outputs


In [ ]:
class bottleNeckPSP(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels, stride, dilation):
        super(bottleNeckPSP, self).__init__()

        self.cbr_1 = conv2DBatchNormRelu(
            in_channels, mid_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        self.cbr_2 = conv2DBatchNormRelu(
            mid_channels, mid_channels, kernel_size=3, stride=stride, padding=dilation, dilation=dilation, bias=False)
        self.cb_3 = conv2DBatchNorm(
            mid_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

        self.cb_residual = conv2DBatchNorm(
            in_channels, out_channels, kernel_size=1, stride=stride, padding=0, dilation=1, bias=False)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        conv = self.cb_3(self.cbr_2(self.cbr_1(x)))
        residual = self.cb_residual(x)
        return self.relu(conv + residual)

In [ ]:
class bottleNeckIdentifyPSP(nn.Module):
    def __init__(self, in_channels, mid_channels, stride, dilation):
        super(bottleNeckIdentifyPSP, self).__init__()

        self.cbr_1 = conv2DBatchNormRelu(
            in_channels, mid_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        self.cbr_2 = conv2DBatchNormRelu(
            mid_channels, mid_channels, kernel_size=3, stride=1, padding=dilation, dilation=dilation, bias=False)
        self.cb_3 = conv2DBatchNorm(
            mid_channels, in_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        conv = self.cb_3(self.cbr_2(self.cbr_1(x)))
        residual = x
        return self.relu(conv + residual)

# Pyramid Pooling 모듈 설명 및 구현

1. Pyramid Pooling 모듈의 서브 네트워크 구성을 이해한다.
2. Pyramid Pooling 모듈의 멀티 스케일 처리 구현 방법을 이해한다.
3. Pyramid Pooling 모듈을 구현할 수 있다.

Pyramid Pooling 모듈의 입력은 Feature 모듈에서 출력된 크기 2048by60by60의 텐서이다. 이 입력이 다섯 개로 분기된다.

가장 위의 분기는 Adaptive Average Pooling 층 (출력=6)으로 보내진다. Adaptive Average Pooling 층은 출력을 통해 화상을 지정된 크기로 변환되도록 Average Pooling을 한다.

다섯 분기 중 네 개는 출력이 각각 6, 3, 2, 1인 Adaptive Average Pooling 층으로 보내진다.

이렇게 출력되는 특징량 맵의 크기가 서로 다른 Average Pooling 층을 사용하면서 원래 입력 화상에 다양한 크기의 특징량 처리를 한다.

Average Pooling 층을 통과한 텐서는 convBatchNormRelu 클래스를 지나 UpSample 층에 도달한다. UpSample 층에서는 Average Pooling 층으로 작아진 특징량의 크기를 Pyramid Pooling 모듈의 입력 크기와 같은 60by60 크기로 확대한다.

In [ ]:
class PyramidPooling(nn.Module):
    def __init__(self, in_channels, pool_sizes, height, width):
        super(PyramidPooling, self).__init__()

        # forward에서 사용화는 화상 크기
        self.height = height
        self.width = width

        # 각 합성곱 층의 출력 채널 수
        out_channels = int(in_channels / len(pool_sizes))

        # pool_sizes = [6, 3, 2, 1]
        self.avpool_1 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[0])
        self.cbr_1 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

        self.avpool_2 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[1])
        self.cbr_2 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

        self.avpool_3 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[2])
        self.cbr_3 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

        self.avpool_4 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[3])
        self.cbr_4 = conv2DBatchNormRelu(
            in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

    def forward(self, x):

        out1 = self.cbr_1(self.avpool_1(x))
        out1 = F.interpolate(out1, size=(
            self.height, self.width), mode="bilinear", align_corners=True)

        out2 = self.cbr_2(self.avpool_2(x))
        out2 = F.interpolate(out2, size=(
            self.height, self.width), mode="bilinear", align_corners=True)

        out3 = self.cbr_3(self.avpool_3(x))
        out3 = F.interpolate(out3, size=(
            self.height, self.width), mode="bilinear", align_corners=True)

        out4 = self.cbr_4(self.avpool_4(x))
        out4 = F.interpolate(out4, size=(
            self.height, self.width), mode="bilinear", align_corners=True)

        output = torch.cat([x, out1, out2, out3, out4], dim=1)

        return output


# Decoder, AuxLoss 모듈 설명 및 구현:

1. Decoder 모듈의 서브 네트워크 구성을 이해한다.
2. Decoder 모듈을 구현할 수 있다.
3. AuxLoss 모듈의 서브 네트워크 구성을 이해한다.
4. AuxLoss 모듈을 구현할 수 있다.

# Decoder 및 AuxLoss 모듈 구조

Decoder 및 AuxLoss 모듈은 Pyramid Pooling 또는 Feature 모듈에서 출력된 텐서 정보를 Decode(읽기)한다. 텐서 정보를 읽은 후 픽셀별로 물체 라벨을 클래스 분류로 추정하고 마지막으로 화상 크기를 원래의 475by475로 업샘플링한다.

출력 텐선느 화상의 각 픽셀에 21 종류 클래스의 확률이 대응하는 값(신뢰도) 입니다. 해당 값이 가장 큰 클래스가 그 픽셀이 속하는 것으로 예측된 물체 라벨이다.

In [ ]:
class DecodePSPFeature(nn.Module):
    def __init__(self, height, width, n_classes):
        super(DecodePSPFeature, self).__init__()

        # forwardで使用する画像サイズ
        self.height = height
        self.width = width

        self.cbr = conv2DBatchNormRelu(
            in_channels=4096, out_channels=512, kernel_size=3, stride=1, padding=1, dilation=1, bias=False)
        self.dropout = nn.Dropout2d(p=0.1)
        self.classification = nn.Conv2d(
            in_channels=512, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = self.cbr(x)
        x = self.dropout(x)
        x = self.classification(x)
        output = F.interpolate(
            x, size=(self.height, self.width), mode="bilinear", align_corners=True)

        return output


In [ ]:
class AuxiliaryPSPlayers(nn.Module):
    def __init__(self, in_channels, height, width, n_classes):
        super(AuxiliaryPSPlayers, self).__init__()

        # forwardで使用する画像サイズ
        self.height = height
        self.width = width

        self.cbr = conv2DBatchNormRelu(
            in_channels=in_channels, out_channels=256, kernel_size=3, stride=1, padding=1, dilation=1, bias=False)
        self.dropout = nn.Dropout2d(p=0.1)
        self.classification = nn.Conv2d(
            in_channels=256, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = self.cbr(x)
        x = self.dropout(x)
        x = self.classification(x)
        output = F.interpolate(
            x, size=(self.height, self.width), mode="bilinear", align_corners=True)

        return output


마지막 클래스 분류 시 전결합 충을 사용하지 않고 클래스 수와 동일하게 21을 출력 채널로 하는 커널 크기 1의 합성곱 층을 사용하는 것이 특징이다. PSPNet은 마지막 클래스 분류에서 전결합 층이 아닌 커널 크기 1의 합성곱 층을 사용한다.

In [ ]:
net = PSPNet(n_classes=21)
net

In [ ]:

batch_size = 2
dummy_img = torch.rand(batch_size, 3, 475, 475)

outputs = net(dummy_img)
print(outputs)